In [5]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import os

### Výběr modelu
1) Darkface_mobilenet
2) Darkface_resnet
3) Widerface_mobilenet
4) Widerface_resnet 

In [6]:
models = ["", "DARKFACE_mobilenet", "DARKFACE_resnet", "WIDERFACE_mobilenet", "WIDERFACE_resnet"]
model_index = 1

### Nastavení thresholdu pro confidence detekce

In [7]:
threshold = 0.25

In [8]:
model_tf = tf.saved_model.load(f"../../gui/models/{models[model_index]}/default/saved_model")

def detect_faces_tf(image):
    """Detekuje obliceje v obrazku

    Args:
        image (np.darray): numpy reprezentace obrazku pro detekci 

    Returns:
        bboxy obliceju (pascalvoc), confidence detekce
    """
    
    
    input_tensor = tf.convert_to_tensor(image)
    
    input_tensor = input_tensor[tf.newaxis, ...]
    detections = model_tf(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                for key, value in detections.items()}
    detections['num_detections'] = num_detections
    
    #print(detections.keys())
    
    return detections['detection_boxes'], detections["detection_scores"]

2023-04-04 17:16:52.457731: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-04 17:16:52.618735: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-04 17:16:52.618853: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-04 17:16:52.621098: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow 

In [9]:
def apply_bboxes(image, bboxes, conf, threshold=0.5, color = (0,0,255)):
    """ Prida do framu bounding boxy 

    Args:
        image (np.darray): zpracovavany frame ve vychozim rozliseni
        bboxes (np.darray): bboxy z funkce detect_faces(_tf)
        conf (np.darray): pole confidence pro jednotlive bboxy z funkce detect_faces(_tf)
        threshold (prah conf pro zobrazeni, optional): prahova hodnota verohodnosti. Defaults to 0.5.
        color (tuple, optional): barva bboxu

    Returns:
        image: puvodni obrazek jako np array s pridanymi bboxy
        nof_detections: pocet detekovanych obliceju, 
        output_bboxes: souradnice ve formatu coco (xywh) pro vystup do txt
    """
    
    
    nof_detections = 0
    output_bboxes = []
    # iterace nad bboxy a zakresleni obdelniku a conf
    for i, box in enumerate(bboxes):
        #print(box)
        # tato cast vychazi z https://hub.gke2.mybinder.org/user/openvinotoolkit-nvino_notebooks-59sp5s7q/notebooks/notebooks/004-hello-detection/004-hello-detection.ipynb
        if(conf[i] > threshold):
            nof_detections+=1
            box = [int(x) for x in box]
            (x_min, y_min, x_max, y_max) = box[1], box[0], box[3], box[2]
            image = cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 2)
                      
            image = cv2.putText(
                    image,
                    f"{float(conf[i]):.2f}",
                    (x_min, y_min - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.8,
                    (255,0,0),
                    1,
                    cv2.LINE_AA,
                )
            
            
            # konec castecne prevzate casti
            output_bboxes.append([x_min, y_min, x_max - x_min, y_max-y_min])
            
    return image, nof_detections, output_bboxes

In [11]:
import json
IMAGES_PATH = "../../api/data/workspace/aug/val/images/"
OUTPUT_PATH = f"results/TF/result_{models[model_index]}_{threshold}.json"

images = sorted(os.listdir(IMAGES_PATH), key=lambda x: int(x.split(".")[0]))

results = []

index = 1
ann_index = 1
for img in images:
    print("Processing file " + str(img))
    
    image = cv2.imread(os.path.join(IMAGES_PATH, img))
    height, width, _ = image.shape
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    bboxes, conf = detect_faces_tf(image)
    
    bboxes[:,1::2] *= image.shape[1] # width
    bboxes[:,::2] *= image.shape[0] #height
    
    image,nof,out_bboxes = apply_bboxes(image, bboxes, conf, threshold=threshold)
    
            
    for n in range(nof):
        data = {
            "image_id": index,
            "category_id": 1,
            "bbox": out_bboxes[n],
            "score": 1.0, 
        }
        results.append(data)
            

    
    index +=1

with open(OUTPUT_PATH, "w") as f:
    json.dump(results, f)

Processing file 1.0.jpg


2023-04-04 17:17:24.206462: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-04-04 17:17:28.761102: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.20GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-04-04 17:17:28.761248: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.20GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-04-04 17:17:28.789921: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.73GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains

Processing file 2.0.jpg
Processing file 3.0.jpg
Processing file 4.0.jpg
Processing file 5.0.jpg
Processing file 6.0.jpg
Processing file 7.0.jpg
Processing file 8.0.jpg
Processing file 9.0.jpg
Processing file 10.0.jpg
Processing file 11.0.jpg
Processing file 12.0.jpg
Processing file 13.0.jpg
Processing file 14.0.jpg
Processing file 15.0.jpg
Processing file 16.0.jpg
Processing file 17.0.jpg
Processing file 18.0.jpg
Processing file 19.0.jpg
Processing file 20.0.jpg
Processing file 21.0.jpg
Processing file 22.0.jpg
Processing file 23.0.jpg
Processing file 24.0.jpg
Processing file 25.0.jpg
Processing file 26.0.jpg
Processing file 27.0.jpg
Processing file 28.0.jpg
Processing file 29.0.jpg
Processing file 30.0.jpg
Processing file 31.0.jpg
Processing file 32.0.jpg
Processing file 33.0.jpg
Processing file 34.0.jpg
Processing file 35.0.jpg
Processing file 36.0.jpg
Processing file 37.0.jpg
Processing file 38.0.jpg
Processing file 39.0.jpg
Processing file 40.0.jpg
Processing file 41.0.jpg
Processi